In [2]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os
import requests
from matplotlib.backends.backend_pdf import PdfPages

# Introduction to python for hydrologists &mdash; pandas


## create a url to read in a single station

In [3]:
url='http://waterservices.usgs.gov/nwis/gwlevels/?format=rdb,1.0&sites=430429089230301&startDT=1880-01-01&endDT=2018-01-01&siteType=GW'

In [4]:
dv_file = requests.get(url)

with open(os.path.join('data',"430429089230301.dat"), 'w') as ofp:
    for carp in dv_file:
        ofp.write(carp.decode())

In [5]:
import os
import numpy as np
NWISfilename = os.path.join('data',"430429089230301.dat")
reconnoiter = open(NWISfilename, 'r').readlines()
for i in np.arange(60):
    print (reconnoiter[i].rstrip())

# ---------------------------------- WARNING ----------------------------------------
# Some of the data that you have obtained from this U.S. Geological Survey database may not
# have received Director's approval.  Any such data values are qualified as provisional and
# are subject to revision.  Provisional data are released on the condition that neither the
# USGS nor the United States Government may be held liable for any damages resulting from its use.
# Additional info: http://help.waterdata.usgs.gov/policies/provisional-data-statement
#
# File-format description:  http://help.waterdata.usgs.gov/faq/about-tab-delimited-output
# Automated-retrieval info: http://help.waterdata.usgs.gov/faq/automated-retrievals
#
# Contact:   gs-w_support_nwisweb@usgs.gov
# retrieved: 2019-11-20 10:43:32 -05:00	(natwebvaas01)
#
# US Geological Survey groundwater levels
#
# Data for the following 1 site(s) are contained in this file
#    USGS 430429089230301 DN-07/09E/23-0005
# -----------------------

In [6]:
numhash = 0 #let's use the as the counter
for line in reconnoiter:
    if line.startswith('#'):
        numhash +=1
    else:
        break
        
print (numhash)

49


# Read in a time series of groundwater levels

In [7]:
colnames = reconnoiter[numhash].rstrip().split()

In [8]:
nwis_df = pd.read_csv(url,sep='\t',
                          skiprows = numhash+2,
                          names = colnames,
                          parse_dates = True,
                          index_col = 3)

In [9]:
nwis_df.head()

,agency_cd,site_no,site_tp_cd,lev_tm,lev_tz_cd,lev_va,sl_lev_va,sl_datum_cd,lev_status_cd,lev_agency_cd
lev_dt,,,,,,,,,,
1946-07-21,USGS,430429089230301,GW,NaN,UTC,105.28,NaN,NaN,NaN,NaN
1946-08-12,USGS,430429089230301,GW,NaN,UTC,102.91,NaN,NaN,NaN,NaN
1946-09-11,USGS,430429089230301,GW,NaN,UTC,102.28,NaN,NaN,NaN,NaN
1946-11-13,USGS,430429089230301,GW,NaN,UTC,102.05,NaN,NaN,NaN,NaN
1947-01-07,USGS,430429089230301,GW,NaN,UTC,101.63,NaN,NaN,NaN,NaN


In [11]:
# This looks like Julian dates to me! But others say Julian dates should actually be days since 1970 or soemthing?
nwis_df.index.dayofyear

Int64Index([202, 224, 254, 317,   7,  46,  77,  83, 104, 132,
            ...
            337,   4,  32,  61,  94, 123, 186, 248, 310, 338],
           dtype='int64', name='lev_dt', length=4086)

In [25]:
# This takes some finagling - julian.to_jd can only take a single value and must be a datetime, not a datetime index
import julian
julian_index = julian.to_jd(pd.to_datetime(nwis_df.index.values[0]), fmt = 'jd')
julian_index

2432022.5

In [30]:
nwis_df.index = [julian.to_jd(i, fmt = 'jd') for i in pd.to_datetime(nwis_df.index.values)]
nwis_df.head()

,agency_cd,site_no,site_tp_cd,lev_tm,lev_tz_cd,lev_va,sl_lev_va,sl_datum_cd,lev_status_cd,lev_agency_cd
2440587.5,USGS,430429089230301,GW,NaN,UTC,105.28,NaN,NaN,NaN,NaN
2440587.5,USGS,430429089230301,GW,NaN,UTC,102.91,NaN,NaN,NaN,NaN
2440587.5,USGS,430429089230301,GW,NaN,UTC,102.28,NaN,NaN,NaN,NaN
2440587.5,USGS,430429089230301,GW,NaN,UTC,102.05,NaN,NaN,NaN,NaN
2440587.5,USGS,430429089230301,GW,NaN,UTC,101.63,NaN,NaN,NaN,NaN


## get rid of columns that are all NaN

In [ ]:
nwis_df.dropna(axis=1,thresh=len(nwis_df), inplace=True)

In [ ]:
nwis_df.head()

In [ ]:
nwis_df.lev_va.plot()

## resample

In [ ]:
nwis_df.lev_va.resample('M').mean().plot(style='.')

In [ ]:
nwis_df.lev_va.resample('A').mean().plot(style='.')

## aggregate

In [ ]:
fig = plt.figure(figsize=(12,4))

mean_lev = nwis_df.lev_va.groupby(nwis_df.index.year).mean()
lower_CI = mean_lev - 2*nwis_df.lev_va.groupby(nwis_df.index.year).std()
upper_CI = mean_lev + 2*nwis_df.lev_va.groupby(nwis_df.index.year).std()
ax = mean_lev.plot(style='r.-')
plt.fill_between(lower_CI.index,lower_CI,upper_CI, color='r',alpha = 0.2)

In [ ]:
fig = plt.figure(figsize=(12,4))
nwis_df.lev_va.groupby(nwis_df.index.year).count().plot(kind='bar',rot=45)

## navigate

In [ ]:
nwis_df.loc[nwis_df.index.year<1950].lev_va.plot()

In [ ]:
nwis_df.loc[(nwis_df.index.year<1950) & (nwis_df.index.year>1948)].lev_va.plot()

## set values

In [ ]:
nwis_df.loc[(nwis_df.index.year<1950) & (nwis_df.index.year>1948), 'lev_va'] += 100

In [ ]:
nwis_df.lev_va.plot()

## groupby

In [ ]:
with PdfPages(os.path.join('data','allyears.pdf')) as outpdf:
    for cname,cgroup in nwis_df.groupby(nwis_df.index.year):
        print(cname)
        plt.figure()
        cgroup.lev_va.plot(title=cname)
        outpdf.savefig()
        plt.close('all')
    

In [ ]:
cgroup